<a href="https://colab.research.google.com/github/ryandhjeon/pytorch-exercise/blob/main/cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying MNIST with CNN

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

### Initialize

In [2]:
# Batch size * Channel * Height * Width
inputs = torch.Tensor(1, 1, 28, 28)
print('Tensor size: {}'.format(inputs.shape))

Tensor size: torch.Size([1, 1, 28, 28])


In [3]:
# Input: 1 channel, output: 32 channel, kernel size: 3, padding: 1
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
# Input: 32 channel, output: 64 channel, kernel size: 3, padding: 1
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
# Max pulling
maxpool = nn.MaxPool2d(2)
print(maxpool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


### Modeling

In [6]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [7]:
out = maxpool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [8]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [9]:
out = maxpool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [10]:
out.size(0)

1

In [11]:
out.size(1)

64

In [12]:
out.size(2)

7

In [13]:
out.size(3)

7

In [14]:
# Leave the batch & spread out the rest
out = out.view(out.size(0), -1) 
print(out.shape)

torch.Size([1, 3136])


### Classify MNIST with CNN

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Set the random seed
torch.manual_seed(777)

# If using GPU, set the random seed 
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [16]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [17]:
mnist_train = dsets.MNIST(root='../dataset/', # Download path
                          train=True, # True: Download training data
                          transform=transforms.ToTensor(), # transforms to tensor
                          download=True)

mnist_test = dsets.MNIST(root='../dataset/', # Download path
                         train=False, # False: Download test data
                         transform=transforms.ToTensor(), # transforms to tensor
                         download=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../dataset/MNIST/raw/train-images-idx3-ubyte.gz to ../dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [18]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [19]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 1st floor
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 2nd floor
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # initialize
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten
        out = self.fc(out)
        return out


In [20]:
# CNN Model
model = CNN().to(device)

In [21]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
total_batch = len(data_loader)
print('Total # of batches: {}'.format(total_batch)) # 600 * 100 = 60,000

Total # of batches: 600


In [23]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # Grab by mini batch. X: Mini batch, Y: label.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.22017993
[Epoch:    2] cost = 0.0609648302
[Epoch:    3] cost = 0.0460369103
[Epoch:    4] cost = 0.0367581584
[Epoch:    5] cost = 0.0299611874
[Epoch:    6] cost = 0.0262526181
[Epoch:    7] cost = 0.0207724534
[Epoch:    8] cost = 0.0185365994
[Epoch:    9] cost = 0.0157309473
[Epoch:   10] cost = 0.0134537667
[Epoch:   11] cost = 0.0113987327
[Epoch:   12] cost = 0.0087028807
[Epoch:   13] cost = 0.00862707663
[Epoch:   14] cost = 0.00681571476
[Epoch:   15] cost = 0.00721790036


In [24]:
# MNIST test data
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9840999841690063


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
